In [ ]:
import psycopg2 as ps
import requests
import time

In [ ]:
# Define credentials.
credentials = {'POSTGRES_ADDRESS' : '#',
               'POSTGRES_PORT' : '#',
               'POSTGRES_USERNAME' : '#',
               'POSTGRES_PASSWORD' : '#',
               'POSTGRES_DBNAME' : '#',
               'API_KEY' : '#'}

In [ ]:
# Before running the create_tables function in this file, add two schemas to
# your PostgreSQL database, one named 'fiveminute', the other named 'onehour'.

# Define currency pairs within each exchange and create the names of the 
# tables for each exchange.
coinbase_pro_pairs = ['bch_btc', 'bch_usd', 'btc_usd', 'btc_usdc', 'dash_btc',
                      'dash_usd', 'eos_btc', 'eos_usd', 'etc_usd', 'eth_btc',
                      'eth_usd', 'eth_usdc', 'ltc_btc', 'ltc_usd', 'xrp_btc',
                      'xrp_usd', 'zec_usdc', 'zrx_usd']
bitfinex_pairs = ['bch_btc', 'bch_usd', 'bch_usdt', 'btc_usd', 'btc_usdt', 
                  'dash_btc', 'dash_usd', 'eos_btc', 'eos_usd', 'eos_usdt', 
                  'etc_usd', 'eth_btc', 'eth_usd', 'eth_usdt', 'ltc_btc',
                  'ltc_usd', 'ltc_usdt', 'xrp_btc', 'xrp_usd', 'zec_usd',
                  'zrx_usd']
hitbtc_pairs = ['bch_btc', 'bch_usdt', 'btc_usdc', 'btc_usdt', 'dash_btc',
                'dash_usdt', 'eos_btc', 'eos_usdt', 'etc_usdt', 'eth_btc', 
                'eth_usdc', 'eth_usdt', 'ltc_btc', 'ltc_usdt', 'xrp_btc', 
                'xrp_usdt', 'zec_usdt', 'zrx_usdt']

hitbtc_table_list = ['hitbtc_' + pair for pair in hitbtc_pairs]

bitfinex_table_list = ['bitfinex_' + pair for pair in bitfinex_pairs]

coinbase_pro_table_list = ['coinbase_pro_' + pair for pair in 
                           coinbase_pro_pairs]

# Define create_tables function.
def create_tables(credentials):
    '''Connects to a PostgreSQL database and adds tables to each respective 
    schema.'''

    # Create connection.
    conn = ps.connect(host=credentials['POSTGRES_ADDRESS'],
                      port=credentials['POSTGRES_PORT'],
                      user=credentials['POSTGRES_USERNAME'],
                      password=credentials['POSTGRES_PASSWORD'],
                      database=credentials['POSTGRES_DBNAME'])


    # Create a cursor.
    cur = conn.cursor()
    
    # Define schemas and table_list.
    schemas = ['fiveminute', 'onehour']
    
    table_list = (hitbtc_table_list + bitfinex_table_list + 
                  coinbase_pro_table_list)
    
    # Loop through schemas and table_list.
    for schema in schemas:
        for table_name in table_list:
            cur.execute('''
            CREATE TABLE {schema}.{table_name}
            (closing_time integer,
            open float,
            high float,
            low float,
            close float,
            base_volume float
            );'''.format(schema=schema, table_name=table_name))
            
    # Commit and close. Verify that tables were created successfully.
    conn.commit()
    
    print("Tables created successfully!")
    conn.close()

In [ ]:
create_tables(credentials)

In [ ]:
# supported exchanges
exchanges = {'bitfinex': bitfinex_pairs,
             'coinbase-pro': coinbase_pro_pairs,
             'hitbtc': hitbtc_pairs}

def insert_data(credentials, exchanges, periods=['300','3600']):
    """This function connects to a database and inserts live data from
    cryptowatch API into tables for each exchange/trading pair combination.
    Option to select a period ('60', '300', '3600'; default=['300', '3600'])
    """

    # connect to database
    conn = ps.connect(host=credentials['POSTGRES_ADDRESS'],
                      database=credentials['POSTGRES_DBNAME'],
                      user=credentials['POSTGRES_USERNAME'],
                      password=credentials['POSTGRES_PASSWORD'],
                      port=credentials['POSTGRES_PORT'])
    # create cursor
    cur = conn.cursor()

    # cryptowatch API URL
    base_url = ('https://api.cryptowat.ch/markets/{exchange}/{trading_pair}/'
                'ohlc?apikey={api_key}&periods={period}')

    # cryptowatch api key
    api_key = credentials['API_KEY']

    # iterate through all exchange/trading pair combinations
    for exchange in exchanges:
        for trading_pair in exchanges[exchange]:
            for period in periods:

                # define schemas in database
                if period =='300':
                    schema = 'fiveminute'
                if period =='3600':
                    schema = 'onehour'

                # table name in database
                table_name = ('_'.join(exchange.split('-')) + '_' +
                              trading_pair)
                
                # cryptowatch wants the trading pair without the underscore
                cleaned_trading_pair = trading_pair.replace('_', '')

                # generate url
                url = base_url.format(exchange=exchange,
                                      trading_pair=cleaned_trading_pair,
                                      api_key=api_key,
                                      period=period)
                
                try:
                    # get response
                    response = requests.get(url).json()
                    candles = response['result'][period]
                    
                    # getting timestamps from candles already in database
                    cur.execute('''SELECT closing_time FROM {schema}.
                                {table_name} order by closing_time desc
                                '''.format(schema=schema,
                                           table_name=table_name))
                    results = cur.fetchall()
                    timestamps = [result[0] for result in results]

                    # ignoring final candle, since it is still open
                    completed_candles = candles[:-1]
                    # only add candles if timestamp not in database
                    for candle in completed_candles:
                        if candle[0] not in timestamps:
                            # we don't need 7th value returned by api...
                            new_data = candle[:6]
                            insert_query = ('INSERT INTO '
                                            '{schema}.{table_name} '
                                            '(closing_time, open, high, low, '
                                            'close, base_volume) VALUES (%s, '
                                            '%s, %s, %s, %s, %s)'
                                           ).format(schema=schema,
                                                    table_name=table_name)
                            cur.execute(insert_query, new_data)
                                
                except:
                    pass

    # commit and close
    conn.commit()
    cur.close()

In [ ]:
while True:
    insert_data(credentials, exchanges)